#Ingest and load python jupyter notebook script

1. Imports
2. Currency pairs data
3. Trading dates (from, to)
4. OHLC trades data. 

In [ ]:
#1.Imports

In [115]:
import requests
import json
import numpy as np
import pandas as pd
import datetime
from datetime import date
from datetime import datetime as dt

In [116]:
#2. Currency pairs data

In [292]:
tradableassetpairs = 'https://api.kraken.com/0/public/AssetPairs'
response_pairs = requests.get(tradableassetpairs)
data = response_pairs.text
json_pairs = json.loads(data)
pairs = json_pairs['result']
dataframe_pairs = pd.DataFrame(columns=('altname','wsname','indexofslash','from','to','quote')) #,'indexvertex'))
dataframe_currencies = pd.DataFrame(columns=('currency','indexvertex'))    

#print(pairs)

i = 0
m = 1

for pair in pairs.values():

    indexofslash = pair.get('wsname').index('/')        
    
    dataframe_pairs.loc[i] = pair.get('altname'),pair.get('wsname'),indexofslash,pair.get('wsname')[:indexofslash],pair.get('wsname')[indexofslash+1:],pair.get('quote')
    i = i + 1
    
        
dataframe_currencies['currency'] = pd.concat([dataframe_pairs['from'],dataframe_pairs['to']])

dataframe_currencies = dataframe_currencies.drop_duplicates()

dataframe_currencies['indexvertex'] = dataframe_currencies['currency'].rank(method='dense').astype('int')

dataframe_currencies.reset_index(drop=True)

#print(dataframe_currencies)

dataframe_currencies.to_csv('vertexs.csv',index=False)

#print(union)
    
dataframe_pairs['indexvertexfrom'] = pd.merge(dataframe_pairs,union, how='left',left_on=['from'], right_on=['currency'])['indexvertex']

dataframe_pairs['indexvertexto'] = pd.merge(dataframe_pairs,union, how='left',left_on=['to'], right_on=['currency'])['indexvertex']

#Row count
#print('Row counts (currency pairs): '+str(len(dataframe_pairs.index)))

#Filter array_pairs by quote (ZUSD, ZEUR, ZAUD, ZGBP)
#quotefilter = 'ZAUD'
#array_pairs = dataframe_pairs.loc[array_pairs['quote'] == quotefilter]

#Print DataFrame
#print(dataframe_pairs)

#Save array_pairs dataframe into csv
dataframe_pairs.to_csv('currencypairs.csv',index=False)
    

    currency  indexvertex
0      1INCH            1
2       AAVE            2
8        ACA            3
10       ACH            4
12       ADA            5
..       ...          ...
626      ZRX          213
64       JPY           96
172      AED            7
174      CHF           44
182      CAD           41

[213 rows x 2 columns]


In [290]:
#3.Trading dates (from, to)

In [274]:
#current date
start_date = str(date.today())
#Set days back for retrieving trades data
days_minus = 0 #1 days back

date_formated = datetime.datetime.strptime(start_date, "%Y-%m-%d")

#get days ago date_formated
end_date = date_formated - datetime.timedelta(days=days_minus)

#print end_date unix timestamp
#print(end_date.timestamp())
end_date_unixtimestamp = end_date.timestamp()

print('Date From: '+str(end_date)+', Date To: '+str(start_date))

Date From: 2022-09-21 00:00:00, Date To: 2022-09-23


In [ ]:
#4. OHLC trades data.

In [276]:
ohlcurl = 'https://api.kraken.com/0/public/OHLC?'
unixtime = str(end_date_unixtimestamp)
interval = str(1440)
concatohlcurl = ''

dataframe_ohlc = pd.DataFrame(columns=('from','indexvertexfrom','to','indexvertexto','altname','unixtimestamp','date','open','high','low','close','vwap','volume','count'))

j = 0

for altname in dataframe_pairs['altname']:
    
    concatohlcurl = ohlcurl +'interval=' + interval + '&since=' + unixtime + '&pair=' + altname
    
    #print(altname+'-'+concatohlcurl)

    
    #print(concatohlcurl)
    
    ohlc = requests.get(concatohlcurl)
    tradesdata = ohlc.text
    json_trades = json.loads(tradesdata)
        
    
    try:
        trades = json_trades['result'][altname]
    except:
        trades = ''
    
    #print(trades)
    
    for trade in trades:
        
        #print(trade)
        
        try:
            pairfrom = dataframe_pairs['from'].loc[dataframe_pairs['altname']==altname].iloc[0]
            indexfrom = dataframe_pairs['indexvertexfrom'].loc[dataframe_pairs['altname']==altname].iloc[0]

        except:
            pairfrom = ''
        
        try:
            pairto = dataframe_pairs['to'].loc[dataframe_pairs['altname']==altname].iloc[0]
            indexto = dataframe_pairs['indexvertexto'].loc[dataframe_pairs['altname']==altname].iloc[0]
        except:
            pairto = ''
        
        unixtimestamp = str(trade[0])
        convertedunixtimestampintodate = dt.utcfromtimestamp(trade[0]).strftime('%Y-%m-%d')
        open = trade[1]
        high = trade[2]
        low = trade[3]
        close = trade[4]
        vwap = trade[5]
        volume = trade[6]
        count = str(trade[7])
        
        dataframe_ohlc.loc[j] = pairfrom,indexfrom,pairto,indexto,altname,unixtimestamp,convertedunixtimestampintodate,open,high,low,close,vwap,volume,count
        
        j = j + 1        

print(dataframe_ohlc)

#save dataframe_ohlc to csv
dataframe_ohlc.to_csv('ohlctrades.csv',index=False)

       from  indexvertexfrom   to  indexvertexto   altname unixtimestamp  \
0     1INCH                1  EUR             68  1INCHEUR    1663718400   
1     1INCH                1  EUR             68  1INCHEUR    1663804800   
2     1INCH                1  EUR             68  1INCHEUR    1663891200   
3     1INCH                1  USD            195  1INCHUSD    1663718400   
4     1INCH                1  USD            195  1INCHUSD    1663804800   
...     ...              ...  ...            ...       ...           ...   
1744    ZRX              213  USD            195    ZRXUSD    1663804800   
1745    ZRX              213  USD            195    ZRXUSD    1663891200   
1746    ZRX              213  XBT            202    ZRXXBT    1663718400   
1747    ZRX              213  XBT            202    ZRXXBT    1663804800   
1748    ZRX              213  XBT            202    ZRXXBT    1663891200   

            date        open        high         low       close        vwap  \
0     2